In [3]:
# pip install -r requirements.txt
import numpy as np
import pandas as pd
import os

In [17]:
streaming_pre = './streaming_history_files'
streaming_paths = os.listdir(streaming_pre)
audio_paths = [x for x in streaming_paths if "Audio" in x]
audio_2025_paths = [x for x in audio_paths if "2025" in x]
audio_2025_paths

['Streaming_History_Audio_2025_13.json',
 'Streaming_History_Audio_2024-2025_12.json',
 'Streaming_History_Audio_2025_14.json']

## 2025 Streams

In [43]:
audio_2025_df = pd.DataFrame()
for f in sorted(audio_2025_paths):
  df_temp = pd.read_json(streaming_pre+"/"+f)
  df_temp["stream_date"] = pd.to_datetime(df_temp["ts"]).dt.date
  audio_2025_df = pd.concat([audio_2025_df, df_temp])
audio_2025_df = audio_2025_df[(audio_2025_df["stream_date"]>=pd.to_datetime("2025-01-01").date()) & 
                              (audio_2025_df["stream_date"]<=pd.to_datetime("2025-12-31").date())]
print(audio_2025_df["stream_date"].min())
print(audio_2025_df["stream_date"].max())
print(audio_2025_df.shape)



2025-01-01
2025-10-09
(23322, 24)


In [46]:
audio_2025_df = audio_2025_df[['platform', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 
                               'master_metadata_album_album_name', 'spotify_track_uri', 'reason_start', 'reason_end', 
                               'skipped', 'stream_date']]
audio_2025_df["seconds"] = audio_2025_df["ms_played"]/1000
audio_2025_df.head()

,platform,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri,reason_start,reason_end,skipped,stream_date,seconds
10278,ios,219813,Sunshine Baby,The Japanese House,In the End It Always Does,spotify:track:7ihVeKd4RjiZ3y12nDhPZO,trackdone,trackdone,False,2025-01-01,219.813
10279,ios,168390,How Long,Yana,How Long,spotify:track:4Nj7olHNFww7pswTjw5K69,trackdone,fwdbtn,True,2025-01-01,168.390
10280,ios,23120,If Now Was Then,Maggie Rogers,Don't Forget Me,spotify:track:4FQiOvG6qN55tdzvu37uUN,fwdbtn,endplay,True,2025-01-01,23.120
10281,ios,267653,The Only Exception,Paramore,Brand New Eyes,spotify:track:7JIuqL4ZqkpfGKQhYlrirs,clickrow,endplay,True,2025-01-01,267.653
10282,ios,261960,Decode,Paramore,Decode,spotify:track:1ZLtE9tSJdaUiIJ9YoKHQe,trackdone,trackdone,False,2025-01-01,261.960


In [73]:
minutes_played = audio_2025_df["seconds"].sum()/60
songs_played = audio_2025_df["spotify_track_uri"].count()
unique_songs_played = audio_2025_df.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"]).ngroups
print(minutes_played)
print(songs_played)
print(unique_songs_played)

43564.493116666665
22921
5178


In [87]:
most_played_songs = audio_2025_df.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"])["master_metadata_track_name"]\
                         .count().reset_index(name="plays").sort_values("plays", ascending=False)
print(most_played_songs["plays"].sum())
most_played_songs.head(10)

22921


,master_metadata_track_name,master_metadata_album_artist_name,plays
2858,Nettles,Ethel Cain,104
2108,Illegal,PinkPantheress,89
4322,Tonight,PinkPantheress,86
4152,The Edge,Sydney Ross Mitchell,81
5050,party 4 u,Charli xcx,79
3974,Sunshine Baby,The Japanese House,76
3846,Sports car,Tate McRae,76
3972,Sunshine & Rain...,Kali Uchis,73
2708,Midnight Sun,Zara Larsson,73
927,Crush,Ethel Cain,72


In [88]:
most_played_artists = audio_2025_df.groupby(["master_metadata_album_artist_name"])["master_metadata_track_name"]\
                         .count().reset_index(name="plays").sort_values("plays", ascending=False)
print(most_played_artists["plays"].sum())
most_played_artists.head(10)

22921


,master_metadata_album_artist_name,plays
303,Charli xcx,728
1720,Zara Larsson,641
516,Ethel Cain,586
334,Clairo,584
1247,PinkPantheress,544
1353,Sabrina Carpenter,528
123,Ariana Grande,485
1543,The Marías,480
302,Chappell Roan,381
40,Addison Rae,339


## All-Time Streaming